In [1]:
import torch
import torch.optim as optim
import torch_geometric
from torch.nn.functional import relu, sigmoid, softmax, mse_loss
from torch.nn import Linear, Module, Dropout, MSELoss, CrossEntropyLoss, BatchNorm1d

from tqdm import tqdm

from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool, GATv2Conv

import pandas as pd
import numpy as np

import os
import pickle
import gzip

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
device = 0
device = torch.device("cuda:{}".format(device) if torch.cuda.is_available() else "cpu")

/home/kuangr/inoue019/.conda/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class MultiHeadAttention(Module):
    def __init__(self, hidden_dim, num_heads, dropout):
        super(MultiHeadAttention, self).__init__()

        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        assert hidden_dim % num_heads == 0

        self.query_linear = Linear(hidden_dim, hidden_dim)
        self.key_linear = Linear(hidden_dim, hidden_dim)
        self.value_linear = Linear(hidden_dim, hidden_dim)

        self.output_linear = Linear(hidden_dim, hidden_dim)
        self.dropout = Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim // num_heads])).to(device)

    def forward(self, query, key, value, mask=None):
        batch_size = query.shape[0]

        Q = self.query_linear(query)
        K = self.key_linear(key)
        V = self.value_linear(value)

        Q = Q.view(batch_size, self.num_heads, -1, self.hidden_dim // self.num_heads)
        K = K.view(batch_size, self.num_heads, -1, self.hidden_dim // self.num_heads)
        V = V.view(batch_size, self.num_heads, -1, self.hidden_dim // self.num_heads)
    
        energy = torch.matmul(Q, K.transpose(-2, -1)) / self.scale
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        attention = self.dropout(softmax(energy, dim=-1))

        weighted_matrix = torch.matmul(attention, V)

        weighted_matrix = weighted_matrix.permute(0, 2, 1, 3).contiguous()
        weighted_matrix = weighted_matrix.view(batch_size, -1, self.hidden_dim)
#         weighted_matrix = weighted_matrix.mean(dim=1)
#         weighted_matrix = torch.max(weighted_matrix, dim=1)
        weighted_matrix = weighted_matrix[:, 0, :]

        output = self.output_linear(weighted_matrix)

        return output

In [3]:
class DrugEncoder(torch.nn.Module):
    def __init__(self, num_node_features, num_emb):
        super(DrugEncoder, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_emb)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return x

In [4]:
class ProteinEncoder(torch.nn.Module):
    def __init__(self, num_node_features, num_emb):
        super(ProteinEncoder, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_emb)

    def forward(self, data):
        x, edge_index, edge_attr, batch = data.x, data.edge_index, data.edge_attr, data.batch

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return x

In [5]:
class DTIPredictor(Module):
    def __init__(self, drug_encoder, protein_encoder, hidden_dim=100, num_heads=4, dropout=0.2):
        super().__init__()
        self.drug_encoder = DrugEncoder(5, 100)
        self.protein_encoder = ProteinEncoder(5, 100)
        self.attention = MultiHeadAttention(hidden_dim, num_heads, dropout)
        self.fc_output = Linear(hidden_dim*3, 1)

    def forward(self, drug_data, protein_data):
        x_drug = self.drug_encoder(drug_data)
        x_protein = self.protein_encoder(protein_data)
        attention_output = self.attention(x_drug, x_protein, x_protein)

        feature = torch.cat((x_drug, attention_output, x_protein), dim=1)
        prediction = self.fc_output(feature)

        return (prediction).squeeze(1)

In [6]:
train = pd.read_csv('../BindingDB/train.csv', index_col=0)
val = pd.read_csv('../BindingDB/val.csv', index_col=0)
test = pd.read_csv('../BindingDB/test.csv', index_col=0)

In [7]:
print('Train dim:', train.shape)
print('val dim:', val.shape)
print('test dim:', test.shape)

Train dim: (15979, 3)
val dim: (2300, 3)
test dim: (4472, 3)


In [8]:
with gzip.open('../drug.pkl.gz', 'rb') as f:
    drug = pickle.load(f)

def get_drug_dataloader(drugs, batch_size=100):
    dataset = [drug[i] for i in drugs]
    return DataLoader(dataset, batch_size=batch_size)

def get_protein_dataloader(proteins, batch_size=100, is_cmap=True):
    if is_cmap:
        dataset = [torch.load('../cmap/{}.pt'.format(i)) for i in proteins]
    else:
        dataset = [torch.load('../protein_graphs/{}.pt'.format(i)) for i in proteins]
        
    return DataLoader(dataset, batch_size=batch_size)

In [ ]:
is_cmap = True
batch_size = 100

protein_train_loader = get_protein_dataloader(train['Target_ID'], batch_size, is_cmap)
protein_val_loader = get_protein_dataloader(val['Target_ID'], batch_size, is_cmap)
protein_test_loader = get_protein_dataloader(test['Target_ID'], batch_size, is_cmap)

In [10]:
model = DTIPredictor(DrugEncoder, ProteinEncoder).to(device)
criterion = MSELoss().to(device)
optimizer = getattr(torch.optim, "Adam")(model.parameters(), lr=0.01,)

train_losses = []
train_accs = []
val_losses = []
val_accs = []

for epoch in tqdm(range(10)):

    model.train()
    total_loss = 0
    for drug, protein, true_y in zip(drug_train_loader, protein_train_loader, train_y):
        drug = drug.to(device)
        protein = protein.to(device)
        true_y = true_y.to(device)
        
        optimizer.zero_grad()
        
        output = model(drug, protein)
        loss = criterion(output, true_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    average_loss = total_loss / len(drug_train_loader)
    train_losses.append(average_loss)
    
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for drug, protein, true_y in zip(drug_val_loader, protein_val_loader, val_y):
            drug = drug.to(device)
            protein = protein.to(device)
            true_y = true_y.to(device)            
            
            output = model(drug, protein)
            loss = criterion(output, true_y)
            val_loss += loss.item()
            val_losses.append(val_loss)

        average_val_loss = val_loss / len(drug_val_loader)
        val_losses.append(average_val_loss)
    print(f'Epoch: {epoch+1}, Train loss: {average_loss:.4f}, Validation Loss: {average_val_loss:.4f}')

 10%|█         | 1/10 [00:02<00:24,  2.68s/it]

Epoch: 1, Train loss: 1435.2893, Validation Loss: 45.0459


 20%|██        | 2/10 [00:04<00:18,  2.31s/it]

Epoch: 2, Train loss: 11.4488, Validation Loss: 3.6336


 30%|███       | 3/10 [00:06<00:15,  2.18s/it]

...

Epoch: 8, Train loss: 1.4579, Validation Loss: 1.8996


 90%|█████████ | 9/10 [00:18<00:02,  2.03s/it]

Epoch: 9, Train loss: 1.2789, Validation Loss: 1.6171


100%|██████████| 10/10 [00:20<00:00,  2.10s/it]

Epoch: 10, Train loss: 1.1560, Validation Loss: 1.4257


In [11]:
model.eval()
pred = []
with torch.no_grad():
    test_loss = 0
    for drug, protein, true_y in zip(drug_test_loader, protein_test_loader, test_y):
        drug = drug.to(device)
        protein = protein.to(device)
        true_y = true_y.to(device)            

        output = model(drug, protein)
        loss = criterion(output, true_y)
        test_loss += loss.item()
        pred.append(output.cpu().detach().numpy()) 
        
pred = np.concatenate(pred)

In [12]:
mse_loss(torch.Tensor(pred), torch.Tensor(np.array(test['Y'])))

tensor(1.3372)